In [1]:
from datetime import date
import pandas as pd
from agent_lab.data_connectors.market_data import fetch_fundamentals
from agent_lab.agents.buffett import BuffettAgent
from agent_lab.agents.ackman import AckmanAgent
from agent_lab.ensemble.oversight import OversightAgent

universe = ["AAPL","MSFT","KO","BAC","JNJ","PG"]
fund = fetch_fundamentals(universe)

buff = BuffettAgent()
ack = AckmanAgent()
agents = [buff, ack]

today = date.today()
all_decs = []

# collect
for agent in agents:
    decs, _ = agent.decide(today, universe, fund)
    all_decs.extend(decs)

# oversight
all_decs_dict = {s: [d for d in all_decs if d.symbol == s] for s in universe}
over = OversightAgent(agent_weights={
    "BuffettAgent": 1.0,
    "AckmanAgent": 0.9
})
final_decisions = over.combine(all_decs_dict)

# build table
rows = []
for sym, d in final_decisions.items():
    row = fund.loc[sym].to_dict()
    rows.append({
        "symbol": sym,
        "Company": row.get("Company"),
        "pe_ratio": row.get("pe"),
        "roe": row.get("roe"),
        "debt_to_equity": row.get("debt_to_equity"),
        "free_cashflow": row.get("free_cashflow"),
        "sector": row.get("sector"),
        "Final Score": d.score,
        "Decision": d.action.name,
        "Confidence": d.confidence,
        "Justification": d.rationale
    })

final_table = pd.DataFrame(rows).set_index("symbol")
final_table.to_csv("buffett_ackman_oversight.csv")
print("✅ Oversight results saved to buffett_ackman_oversight.csv")
final_table


✅ Oversight results saved to buffett_ackman_oversight.csv


,Company,pe_ratio,roe,debt_to_equity,free_cashflow,sector,Final Score,Decision,Confidence,Justification
symbol,,,,,,,,,,
AAPL,Apple Inc.,35.089394,1.49814,154.486,94873747456,Technology,0.675,BUY,0.675,BuffettAgent:Action.HOLD(0.50); AckmanAgent:Ac...
MSFT,Microsoft Corporation,38.163610,0.33281,32.661,61070376960,Technology,0.675,BUY,0.675,BuffettAgent:Action.HOLD(0.50); AckmanAgent:Ac...
KO,The Coca-Cola Company,24.794327,0.42370,166.367,1707124992,Consumer Defensive,1.425,BUY,1.000,BuffettAgent:Action.BUY(0.75); AckmanAgent:Act...
BAC,Bank of America Corporation,13.765395,0.09460,0.000,0,Financial Services,0.000,HOLD,0.000,BuffettAgent:Action.HOLD(0.50); AckmanAgent:Ac...
JNJ,Johnson & Johnson,18.912205,0.30212,64.686,11079499776,Healthcare,1.805,BUY,1.000,BuffettAgent:Action.BUY(0.95); AckmanAgent:Act...
PG,The Procter & Gamble Company,23.711212,0.31242,69.861,11380124672,Consumer Defensive,1.425,BUY,1.000,BuffettAgent:Action.BUY(0.75); AckmanAgent:Act...
